We are going to implement the backtest function with numba and extension in cpp using cython

Numby way faster, there must be some implementations details that are making the code slow. probably using lists and append?

In [1]:
from src.core.backtest.backtesting import backtest_numba, backtest_pure_python
from src.core.backtest.backtesting_cpp import backtest_cpp, backtest_cpp_no_list

In [2]:
#Deserialize
import pickle
import pandas as pd
from src.modules.paths import get_project_root
from src.core.features.utils import feature_pipeline, add_domain_features
from sklearn.metrics import r2_score
import matplotlib.pyplot as plt
with open("wow_20.pickle" ,'rb') as f:
    model = pickle.load(f)
import optuna
import numpy as np

In [3]:
cut_off_date ="2021-11-07 21:28:00"
#cut_off_date ="2021-09-07 21:28:00"
#cut_off_date ="2021-10-23 21:28:00"
crypto_to_optimize=["bcheur","compeur","xdgeur","xlmeur"]
crypto_to_evaluate = ["etheur", "algoeur", "bateur", "adaeur","xrpeur"]
columns_features = [col for col in feature_pipeline(pd.read_csv(get_project_root() / "data" / "historical" / "etheur.csv"), include_target=False)[0].columns if col.startswith("feature")]

In [4]:



df_all = pd.read_feather(get_project_root() / "data" / "training" / "training_all_crypto_28_11_2021.feather")
df = df_all[df_all.date>cut_off_date].copy()

In [5]:
df["preds"] = model.predict(df[columns_features])

In [6]:
df_backtest = df.copy()#[df.pair_name=="xlmeur"]

In [7]:
params = {'take_profit_pct': 0.049103743582114695,
  'stop_loss_pct': 0.18220384882120128,
  'lim_pred_buy': -1,
"fee": 0.0016,

"security_factor": 0.0024,

"shift": 1
}

In [13]:
%time actions = backtest_numba(indexes= df_backtest.time.values, open_values=df_backtest.open.values, preds_values=df_backtest.preds.values,  **params)

CPU times: user 1.44 ms, sys: 84 µs, total: 1.52 ms
Wall time: 1.57 ms


In [9]:
%time actions = backtest_pure_python(indexes= df_backtest.time.values, open_values=df_backtest.open.values, preds_values=df_backtest.preds.values,  **params)

CPU times: user 529 ms, sys: 1.61 ms, total: 531 ms
Wall time: 530 ms


In [10]:
%time actions = backtest_cpp(indexes= df_backtest.time.values, open_values=df_backtest.open.values, preds_values=df_backtest.preds.values,  **params)

CPU times: user 75.6 ms, sys: 10.1 ms, total: 85.7 ms
Wall time: 85.6 ms


In [11]:
%time actions = backtest_cpp_no_list(indexes= df_backtest.time.values, open_values=df_backtest.open.values, preds_values=df_backtest.preds.values,  **params)

CPU times: user 86.6 ms, sys: 3.73 ms, total: 90.3 ms
Wall time: 97 ms
